<a href="https://colab.research.google.com/github/MarcoE25/Clases-Analisis-Datos/blob/main/Clase_8_intermedio_202508.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Itinerario de clase

* Tablas dinámicas en Pandas
* Índices
* Filtro avanzado con índices y NumPy
* Funciones avanzadas: ``pipe``, ``assign`` y ``replace``

# Resumen de la clase anterior

In [ ]:
# PONER EN LA PRIMER CELDA TODAS LAS LIBRERÍAS
import pandas as pd
from dateutil import parser
import datetime as dt
import random as r
import numpy as np

# Webscraping
import urllib.request, json
from urllib.request import urlopen
from zipfile import ZipFile
import re
import requests

################################################################################
############################ CARGA DE LA INFORMACION ###########################
################################################################################
df_bank = pd.read_csv("https://cursopypagina.github.io/CursoPy/Clase_1.csv")
df_bank_ej = pd.read_csv("https://cursopypagina.github.io/CursoPy/Clase_1_5.csv")

df_bank_ej = df_bank_ej.rename(columns={"Fecha_Inicio_Contrato": "Fecha_Inicio"})
df_bank_ej = df_bank_ej[["Nombre_Empresa", "Ejecutivo", "Fecha_Inicio"]]

################################################################################
############################ VALORES DEL USD-EUR ###############################
################################################################################
def divisa_webs(clave):
  url = "https://www.piplatam.com/Home/filiales?country=MX"
  page = urlopen(url)
  htmlBytes = page.read()
  html = htmlBytes.decode("utf-8")
  strValorClave = html.find(clave)
  stringFiltro1 = html[strValorClave:]
  strValor = stringFiltro1.split("dblValue")[1].split("dblChange")[0]
  strValor = re.findall(r"\d+\.\d+", strValor)[0]
  return strValor

usdValor = round(float(divisa_webs("USD-MXN FIX (Banxico)")), 2)
eurValor = round(float(divisa_webs("EUR-MXN")), 2)

################################################################################
########################## LIMPIEZA Y PROCESAMIENTO ############################
################################################################################
def clean_1(df_func, tipo):
  df_func_trabajo = df_func.copy()
  df_func_trabajo['Nombre_Empresa'] = df_func_trabajo['Nombre_Empresa'].str.strip().str.title()
  df_func_trabajo["Fecha_Inicio"] = df_func_trabajo["Fecha_Inicio"].apply(lambda x: parser.parse(x))

  if tipo == "Ejecutivo":
    ############################################################################
    # Agregamos la parte donde nos quedamos solo con un ejecutivo de ventas
    df_func_trabajo = df_func_trabajo.groupby("Nombre_Empresa").agg(
        {
            "Fecha_Inicio": ['max'],
            "Ejecutivo": ['last']
        }
    )
    df_func_trabajo = df_func_trabajo.reset_index()
    df_func_trabajo.columns = ['Nombre_Empresa', 'Fecha_Inicio', 'Ejecutivo']
    del df_func_trabajo['Fecha_Inicio']
    return df_func_trabajo
  elif tipo == "General":
    df_func_trabajo['Fecha_Fin'] = df_func_trabajo['Fecha_Fin'].fillna('1900-01-01')
    df_func_trabajo['Plazo'] = df_func_trabajo['Plazo'].fillna(0)
    df_func_trabajo['Plazo'] = df_func_trabajo['Plazo'].astype(int)

    df_func_trabajo['Fecha_Fin'] = df_func_trabajo['Fecha_Fin'].apply(lambda x: parser.parse(x))
    df_func_trabajo['Operacion'] = df_func_trabajo['Operacion'].str.split("-").apply(lambda x: x[1])

    fechaReporte = dt.datetime.today() - dt.timedelta(days = 1)
    fechaReporte = dt.datetime.strftime(fechaReporte, "%Y-%m-%d")
    df_func_trabajo['Fecha_Reporte'] = fechaReporte

    # Columnas condicionales
    zonas = ["Norte", "Sur", "Centro"]
    df_func_trabajo['Zonas'] = [r.choice(zonas) for i in range(len(df_func_trabajo))]

    def valorizador(divisa):
      if divisa == "USD":
        return usdValor
      elif divisa == "EUR":
        return eurValor
      else:
        return 1
    df_func_trabajo['Divisa_Valor'] = df_func_trabajo['Divisa'].apply(valorizador)
    df_func_trabajo['Monto_Credito'] = df_func_trabajo['Monto_Credito'] * df_func_trabajo['Divisa_Valor']
    del df_func_trabajo['Divisa_Valor']

    return df_func_trabajo
  else:
    print("⚠️Nombre inválido⚠️")

################################################################################
##################### APLICAMOS LA FUNCION DE LIMPIEZA #########################
################################################################################
df_bank_clean = clean_1(df_bank, "General")
df_bank_clean_ej = clean_1(df_bank_ej, "Ejecutivo")

# Obtenemos el merge
df_bank_final = df_bank_clean.merge(df_bank_clean_ej, on='Nombre_Empresa', how='left')

## Tablas dinámicas en pandas

### 📌 ¿Qué es?

* Es una función de pandas (pd.pivot_table) que permite reorganizar y agregar la información de un DataFrame.

* Toma columnas como filas (index), otras como columnas (columns) y calcula un valor agregado en las celdas con una función de agregación (aggfunc).

### 📌 ¿Qué hace?

* Agrupa los datos en función de las variables que le digas (ej. Ciudad, Género, Estado_Credito).

* Agrega los valores de otra columna con una función (ej. suma, promedio, conteo, máximo).

* Devuelve una nueva tabla que organiza los resultados de forma resumida y fácil de leer.

### Esqueleto

```python
pd.pivot_table(
    data,
    values=None,     # qué columna(s) quieres analizar
    index=None,      # se convierten en filas
    columns=None,    # se convierten en columnas
    aggfunc="mean",  # función de agregación (mean, sum, count, max, etc.)
    fill_value=None  # reemplazo para valores faltantes
)
```

In [ ]:
# Semilla para reproducibilidad
np.random.seed(42)

# Simulación de datos de créditos bancarios
n = 200

data = {
    "ID_Cliente": np.arange(1, n+1),
    "Genero": np.random.choice(["Masculino", "Femenino"], size=n),
    "Edad": np.random.randint(18, 70, size=n),
    "Ciudad": np.random.choice(["Ciudad de México", "Monterrey", "Guadalajara", "Puebla", "Querétaro"], size=n),
    "Tipo_Credito": np.random.choice(["Hipotecario", "Automotriz", "Personal", "Empresarial"], size=n),
    "Monto_Credito": np.random.randint(5000, 500000, size=n),
    "Tasa_Interes": np.round(np.random.uniform(5, 25, size=n), 2),
    "Estado_Credito": np.random.choice(["Activo", "Pagado", "En Mora"], size=n, p=[0.5, 0.3, 0.2])
}

df = pd.DataFrame(data)
df.head()

,ID_Cliente,Genero,Edad,Ciudad,Tipo_Credito,Monto_Credito,Tasa_Interes,Estado_Credito
0,1,Masculino,49,Guadalajara,Hipotecario,155159,8.60,Activo
1,2,Femenino,56,Guadalajara,Hipotecario,177502,18.93,Activo
2,3,Masculino,66,Monterrey,Hipotecario,160576,13.23,Activo
3,4,Masculino,69,Querétaro,Empresarial,405330,22.49,Pagado
4,5,Masculino,49,Ciudad de México,Automotriz,346003,15.30,Pagado


In [ ]:
# Promedio del monto de crédito por Ciudad y Tipo de Crédito
pd.pivot_table(
    df,
    values="Monto_Credito",
    index="Ciudad",
    columns="Tipo_Credito",
    aggfunc="mean"
)

Tipo_Credito,Automotriz,Empresarial,Hipotecario,Personal
Ciudad,,,,
Ciudad de México,154690.833333,288833.125000,284032.642857,168961.250000
Guadalajara,274754.500000,254608.666667,238721.230769,290938.400000
Monterrey,294430.538462,198463.700000,261787.692308,223215.916667
Puebla,256398.571429,162011.428571,304775.916667,253783.466667
Querétaro,175954.000000,212523.857143,292580.714286,227288.000000


In [ ]:
# Conteo de créditos por Género y Estado del Crédito
pd.pivot_table(
    df,
    values="ID_Cliente",       # usamos el ID solo para contar
    index="Genero",            # filas = Género
    columns="Estado_Credito",  # columnas = Estado del crédito
    aggfunc="count",           # contamos
    fill_value=0
)

Estado_Credito,Activo,En Mora,Pagado
Genero,,,
Femenino,48,19,33
Masculino,52,23,25


In [ ]:
# Tasa de interés promedio por Tipo de Crédito y Ciudad
pd.pivot_table(
    df,
    values="Tasa_Interes",
    index="Ciudad",            # filas = Ciudad
    columns="Tipo_Credito",    # columnas = Tipo de crédito
    aggfunc="mean"             # promedio
)

Tipo_Credito,Automotriz,Empresarial,Hipotecario,Personal
Ciudad,,,,
Ciudad de México,18.580000,18.683750,14.975000,18.632500
Guadalajara,13.036667,13.670000,14.766923,16.658000
Monterrey,17.473077,14.250000,13.125385,15.101667
Puebla,14.635714,16.297143,15.255833,14.557333
Querétaro,13.105000,15.152143,14.852857,16.157778


## Índices

* ``iloc``: Te permite hacer uso y filtrado de información con base en índices.
* ``iloc[i, j]``: accedes a la fila de índice ``i`` y la columna de índice ``j``.
* ``loc``: Te permite hacer uso y filtrado de información con base en índices y etiquetas
* ``loc[i, j]``: accedes a la fila de índice o etiqueta ``i`` y la columna de índice o etiqueta``j``.

index | Nombre (col_indice 0) | Edad (col_indice 1)
---|---|---
0|Luis|17
1|Fernando|24

Entonces, un ejemplo del uso de las funciones ``iloc`` y ``loc`` sería:

```python
# Accedemos al valor de la fila 0 y de la columna de indice 1
print(df.iloc[0, 1]) # 17
print(df.iloc[1, 1]) # 24
print(df.iloc[0, 0]) # Luis

print(df.loc[0, "Nombre"]) # Luis
print(df.loc[1, "Edad"]) # 24
```

los índices son muy útiles cuando queremos iterar sobre un dataframe

In [ ]:
# Recordemos que tenemos los valores de los tipos de cambio
print(usdValor, eurValor)

# y que tenemos el insumo original
display(df_bank.head())

18.48 21.69


,Nombre_Empresa,Monto_Credito,Divisa,Operacion,Fecha_Inicio,Fecha_Fin,Plazo
0,grupo bimbo,4117589.71,MXN,OP-100000,27-Aug-2019,"April 24, 2023",1336.0
1,WALMART DE MÉXICO,7008028.47,USD,OP-100001,2017-04-10,18/02/2022,1775.0
2,liverpool,9918174.32,EUR,OP-100002,"April 21, 2017",NaN,NaN
3,grupo lala,1293043.34,USD,OP-100003,24-Nov-2023,NaN,NaN
4,FEMSA,7128647.73,EUR,OP-100004,2024-10-30,NaN,NaN


podemos valorizar la columna ``Monto_Credito`` iterando sobre el dataframe como sigue:

In [ ]:
# En general, iteraremos sobre el index del dataframe
# El index es la columna numerica que se ve como la primer columna
# del dataframe

# Podemos acceder al indice de un dataframe escribiendo
df_bank.index

RangeIndex(start=0, stop=10000, step=1)

In [ ]:
# asi, valoricemos la columna monto del credito con un bucle y
# con la función loc (se recomienda en estos caso el uso de loc
# sobre el de iloc)

# Creamos un columna nueva
df_bank['Monto_Credito_Valorizado'] = 0

# Recorremos el indice del dataframe
# es decir, estamos recorriendo la numeracion de las filas
# del dataframe
for i in df_bank.index:
  # Accedemos al monto del credito de la fila i
  monto_i = df_bank.loc[i, 'Monto_Credito']

  # Valorizamos dependiendo el tipo de cambio
  divisa_i = df_bank.loc[i, 'Divisa']
  if divisa_i == "USD":
    monto_i_val = monto_i * usdValor
  elif divisa_i == "EUR":
    monto_i_val = monto_i * eurValor
  else:
    monto_i_val = monto_i * 1

  # Rellenamos la informacion de la columna nueva
  # insertando el valor obtenido i en la fila i
  # correspondiente
  df_bank.loc[i, 'Monto_Credito_Valorizado'] = monto_i_val

df_bank[['Monto_Credito', 'Divisa', 'Monto_Credito_Valorizado']]

,Monto_Credito,Divisa,Monto_Credito_Valorizado
0,4117589.71,MXN,4.117590e+06
1,7008028.47,USD,1.295084e+08
2,9918174.32,EUR,2.151252e+08
3,1293043.34,USD,2.389544e+07
4,7128647.73,EUR,1.546204e+08
...,...,...,...
9995,7649613.74,MXN,7.649614e+06
9996,1578366.47,MXN,1.578366e+06
9997,2057031.71,MXN,2.057032e+06
9998,6555258.51,MXN,6.555259e+06


# Filtro avanzado con índices y NumPy

In [ ]:
# Para eliminar filas dentro de un dataframe, en esencia
# 1. Los indices son identificadores del dataframes
# 2. Realizamos un filtro de índices
# 3. Quitamos esos indices del dataframe a considerar